In [9]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException



opcoes_google = Options()
#opcoes_google.add_argument("--headless")  # Ativa o modo headless
opcoes_google.add_argument("--disable-gpu")  # Desativa uso de GPU (opcional para estabilidade)
opcoes_google.add_argument("--window-size=1920,1080")  # Define tamanho da janela no modo headless
opcoes_google.add_argument("--no-sandbox")  # Útil em ambientes de servidor
opcoes_google.add_argument("--disable-dev-shm-usage")  # Evita problemas de memória compartilhada


#le a planilha moedas onde ira ler a lista de moedas inseridas
planilha  = pd.read_excel("Moedas.xlsx")

# Caminho para o chromedriver
driver_path = r"C:\Users\Vitor Cavaco\Documents\RPA\COTACAO_DOLAR\chromedriver.exe"

# Crie uma instância de Service passando o caminho do driver
service = Service(driver_path)

# Inicie o ChromeDriver com o serviço e as opções de download
driver = webdriver.Chrome(service=service, options=opcoes_google)

# Acessa o site
driver.get("https://wise.com/br/currency-converter/eur-to-brl-rate?amount=1")

# Aguarda o campo de busca estar disponível
wait = WebDriverWait(driver, 50)

#Estrutura para ler as moedas da planilha
for moeda in planilha['moeda']:
    
    #Estrutura para caso dê erro de StaleElementReferenceException 
    for aguarde in range(3):

        try:
            seleciona = wait.until(EC.element_to_be_clickable((By.ID,'source-inputSelectedCurrency')))
            seleciona.click()
            break
        
        except StaleElementReferenceException:
            
            print(f'Falha na {aguarde + 1} tentativa')

    #Seleciona o campo de busca 
    nome_moeda = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="source-inputSelectedCurrencySearch"]')))
    nome_moeda.send_keys(moeda)

    #Interage clicando 
    clica = wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'np-select-input-listbox')))
    clica.click()

    #Recebe o atributo VALUE do HTML
    valor = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="target-input"]')))
    valor_moeda = valor.get_attribute("value")

    print(f"{moeda} : R$: {valor_moeda}")

driver.quit()

print("Codigo Finalizado...")
   

    

USD : R$: 6,21
euro : R$: 6,41
Libra esterlina : R$: 7,73
Iene : R$: 0,04
Dólar Australiano : R$: 3,86
Franco Suíço : R$: 6,85
Dólar Canadense : R$: 4,31
Peso Argentino : R$: 0,01
Lira Turca : R$: 0,18
Codigo Finalizado...
